In [1]:
import pandas as pd
import pickle
import numpy as np
import re 
import datetime as dt

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
# Read both datasets
os_kb_entities=pd.read_csv(f'open_sanctions_entities.csv',index_col=0)
os_kb_entities['kb_origin']='open_sanctions'
ls_kb_entities=pd.read_csv(f'lilsis_entities.csv',index_col=0)
ls_kb_entities['kb_origin']='lilsis'

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# Rename columns to concat datasets  
os_kb_entities=os_kb_entities.rename(columns={'full_notes':'desc'})
ls_kb_entities=ls_kb_entities.rename(columns={'primary_ext':'schema','end_date':'deathdate','aliases':'AKA','start_date':'birthdate','context':'desc'})

In [6]:
# Concat datasets into one
kb_entities=pd.concat([os_kb_entities,ls_kb_entities]).reset_index().rename(columns={'index':'original_index'})

In [7]:
# Drop useless columns
kb_entities.drop(['schema','notes','context','types','start_date_sentence','end_date_sentence'],1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [8]:
kb_entities.shape

(503310, 11)

In [9]:
# Fix trailing whitespaces
kb_entities['desc']=kb_entities['desc'].apply(lambda x: re.sub(r"\b(\.)[\.\s]+$", "\\1", x))

In [10]:
# Remove entities with no description
kb_entities=kb_entities[kb_entities['desc'].str.replace(' ','').apply(len)>0]

In [11]:
# Drop duplicated based on same name and description
kb_entities.drop_duplicates(subset=['name','desc'],inplace=True)

In [12]:
kb_entities.shape

(429953, 11)

In [13]:
## Resolve duplicate entitiy IDs

In [14]:
# Find duplicate entries on the 'id' columns
redundant_entities_by_id=kb_entities[kb_entities['id'].duplicated(keep=False)].sort_values(['id','name'])

In [15]:
# Drop duplicate_entities (these were all cases where the ID was taken from Wikidata)
redundant_entities_indices=redundant_entities_by_id.index
kb_entities.drop(redundant_entities_indices, inplace=True)

In [16]:
# Keep first of duplicated entities
redundant_entities_by_id_consolidated_desc=redundant_entities_by_id.groupby(['id']).first().reset_index()

In [17]:
# Concatenate back to kb entity dataframe 
kb_entities=pd.concat([kb_entities,redundant_entities_by_id_consolidated_desc])
kb_entities.reset_index(drop=True,inplace=True)

In [18]:
kb_entities.shape

(429861, 11)

In [19]:
## Resolve duplicates on birthdate, deathdate and website

In [20]:
# Create auxiliary column for ordering based on len of desc field
kb_entities['desc_len']=kb_entities['desc'].str.len()

In [21]:
redundancy_cols=['birthdate','deathdate','website']
for col in redundancy_cols:
    # Find duplicates ordered by description len
    redundant_entities_by_col=kb_entities[~(kb_entities[col].isna())&
                (kb_entities.duplicated(['name',col],keep=False))
               ].sort_values(by=['name','desc_len'],ascending=False)

    # Drop duplicate_entities on name and birthdate
    redundant_entities_indices=redundant_entities_by_col.index
    kb_entities.drop(redundant_entities_indices, inplace=True)

    # Keep first of duplicated entities
    redundant_entities_by_col_consolidated_desc=redundant_entities_by_col.groupby(['name',col]).first().reset_index()

    # Concatenate back to kb entity dataframe 
    kb_entities=pd.concat([kb_entities,redundant_entities_by_col_consolidated_desc])
    kb_entities.reset_index(drop=True,inplace=True)

In [22]:
kb_entities.shape

(428519, 12)

In [23]:
## Further duplicate inspection 

In [24]:
# Duplicates on name
kb_duplicated=kb_entities[kb_entities.duplicated(keep=False,subset=['name'])].sort_values(by=['name'])

In [25]:
kb_entities.shape

(428519, 12)

In [26]:
# Standardise names 
kb_entities['name']=kb_entities['name'].str.title()

In [27]:
# Ensure person name is included in all descriptions
name_not_in_notes_indices=kb_entities[kb_entities.apply(lambda x: x['name'].lower() not in x['desc'].lower(),axis=1)].index.values
naming_string = 'This person is called '
kb_entities.loc[name_not_in_notes_indices,'desc'] = kb_entities.loc[name_not_in_notes_indices].apply(lambda x: naming_string + x['name'] + '. ' + x['desc'],axis=1)

In [28]:
# Ensure descriptions end in stop mark. 
kb_entities.loc[~kb_entities['desc'].isna(),'desc']=kb_entities.loc[~kb_entities['desc'].isna(),'desc'].apply(lambda x: x + '.' if x[-1]!='.' else x)

In [29]:
# Clean up data by removing multiple trailing stop marks
multi_stopmarks_expr = re.compile('\.\s?\.')

for expr in [multi_stopmarks_expr]:
    # Replace expression in string
    kb_entities.loc[~kb_entities['desc'].isna(),'desc']=kb_entities.loc[~kb_entities['desc'].isna(),'desc'].apply(lambda x: ''.join([re.sub(expr, '. ', x)]))

In [30]:
## Add KB URLs

In [31]:
def url_generator(id_,name, dataset):
    if dataset=='open_sanctions':
        return f'https://www.opensanctions.org/entities/{id_}'
    if dataset=='lilsis':
        return f'https://littlesis.org/person/{id_}-{name}'

In [32]:
kb_entities['kb_url']=kb_entities.apply(
    lambda x: [url_generator(x['id'],x['name'], 'open_sanctions') 
                             if x['kb_origin']=='open_sanctions' 
                             else url_generator(x['id'],x['name'], 'lilsis') 
                                                  ][0],1
)

In [33]:
dataset='full'
kb_iteration=dt.datetime.now().strftime('%Y_%m_%d')
dataset=f'{dataset}_{kb_iteration}'
kb_entities.to_csv(f'kb_entities_{dataset}.csv')

In [34]:
data_26=pd.read_csv('kb_entities_full_2022_10_26.csv')
data_07=pd.read_csv('kb_entities_full_2022_11_07.csv')

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
data_26.shape

(428519, 14)

In [36]:
data_07.shape

(428519, 14)

In [41]:
merge=data_26[['id','desc']].merge(data_07[['id','desc']],on=['id'],how='outer')

In [42]:
merge[(merge['desc_x']!=merge['desc_y'])]

,id,desc_x,desc_y
0,acf-00040861bc3f593000830d987d09967ef3503ef1,"Kolyvanov Egor is a Russian propagandist: host of news program ""Segodnia"" (""Today"") on NTV. This person has held these positions: Propagandists, Federal media employees. This person is a male. This person was born in 1980-11-15.","Kolyvanov Egor is a Russian propagandist: host of news program ""Segodnia"" (""Today"") on NTV. This person has held these positions: Federal media employees, Propagandists. This person is a male. This person was born in 1980-11-15."
3,acf-002c208139012c8d93b6298358188d7cadafe648,"Goreslavsky Alexey Sergeyevich is a Russian journalist and media manager. Helped destroy independent media in Russia when he was appointed the new editor-in-chief of the previously popular independent Lenta.ru in 2014 . Russian propagandist: journalist and media manager; works at the state-owned Institute for the Development of Internet that focuses on creating propoganda for the youth of Russia Head of the Internet Development Institute. Responsible for organizing censorship in the Russian segment of the Internet. This person has held these positions: Propagandists, Investigations subjects, Internet censors, Federal media employees, Individuals involved in corruption, Organizers of political repressions. This person is a male. This person was born in 1977-07-13.","Goreslavsky Alexey Sergeyevich is a Russian journalist and media manager. Helped destroy independent media in Russia when he was appointed the new editor-in-chief of the previously popular independent Lenta.ru in 2014 . Russian propagandist: journalist and media manager; works at the state-owned Institute for the Development of Internet that focuses on creating propoganda for the youth of Russia Head of the Internet Development Institute. Responsible for organizing censorship in the Russian segment of the Internet. This person has held these positions: Individuals involved in corruption, Organizers of political repressions, Federal media employees, Investigations subjects, Internet censors, Propagandists. This person is a male. This person was born in 1977-07-13."
4,acf-002cc8fdf8fe41185091a7cb6c598663e7a22eb5,"Samoilova Natalya Vladimirovna is a Russian singer, composer. Supported the actions of the Russian military during the invasion of Ukraine. This person has held these positions: Celebrities influencers and bloggers, Sellout opinion leaders. This person is a female. This person was born in 1987-06-24.","Samoilova Natalya Vladimirovna is a Russian singer, composer. Supported the actions of the Russian military during the invasion of Ukraine. This person has held these positions: Sellout opinion leaders, Celebrities influencers and bloggers. This person is a female. This person was born in 1987-06-24."
12,acf-00776ee02694fb36ef3d6f42fa95ddcdab7f76c8,"Vasilev Vladimir Lvovich is a Director and Editor-in-Chief of the newspaper ""Sovetskaia Chuvashiia"" (owned by the Government of the Chuvash Republic). This person has held these positions: Propagandists, Regional press. This person is a male.","Vasilev Vladimir Lvovich is a Director and Editor-in-Chief of the newspaper ""Sovetskaia Chuvashiia"" (owned by the Government of the Chuvash Republic). This person has held these positions: Regional press, Propagandists. This person is a male."
13,acf-008076c876c08dd7c54a05cbec8f82eeaa0af206,"Semchenkov Iurii Evgenevich is a Editor-in-Chief of the newspaper ""Smolenskaia gazeta"" (owned by the Government of the Smolensk region). This person has held these positions: Propagandists, Regional press. This person is a male.","Semchenkov Iurii Evgenevich is a Editor-in-Chief of the newspaper ""Smolenskaia gazeta"" (owned by the Government of the Smolensk region). This person has held these positions: Regional press, Propagandists. This person is a male."
...,...,...,...
428463,Q98819671,"Vadim Dmitrievich Ipatov is a Deputy Chairperson, Central Electoral Commission (CEC). As a Member of the CEC, he was resp